**提供资料：**
数据所在路径：/content/drive/MyDrive/深度学习100例/Pytorch版/01cat-dog.zip

步骤：数据处理-搭建模型-设置损失函数和优化器-模型训练-模型评估

In [ ]:
!unzip '/content/drive/MyDrive/深度学习100例/Pytorch版/01cat-dog.zip'

## 1.数据预处理

步骤：

**处理数据-读取数据-包装数据**

1.  transforms.compose
2.  datasets.ImageFolder
3.  torch.utils.data.DataLoader

In [2]:
train_dir='/content/1-cat-dog/train'
test_dir='/content/1-cat-dog/val'

In [3]:
import torch
from torch import nn
import torchvision.transforms as transforms

In [4]:
train_transforms=transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])
test_transforms=transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])

In [5]:
from torchvision import datasets

In [6]:
train_data=datasets.ImageFolder(root=train_dir,transform=train_transforms)
test_data=datasets.ImageFolder(root=test_dir,transform=test_transforms)

In [7]:
from torch.utils.data import DataLoader

In [8]:
train_loader=DataLoader(train_data,batch_size=4,shuffle=True,num_workers=1)
test_loader=DataLoader(test_data,batch_size=4,shuffle=True,num_workers=1)

## 2.定义网络模型

![LeNet structure](https://www.datasciencecentral.com/wp-content/uploads/2021/10/1gNzz6vvWmF6tDN6pTRTd9g.jpeg)

In [9]:
import torch.nn.functional  as F

In [10]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [11]:
print("Using %s device"%device)

Using cpu device


In [12]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*53*53,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,2)

  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.pool(x)
    x=F.relu(self.conv2(x))
    x=self.pool(x)

    x=x.view(-1,16*53*53)

    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

model=LeNet().to(device)
print(model)



LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


# 3.定义损失函数和优化器

In [13]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

# 4.定义训练函数

训练函数需要哪些东西？

数据:dataloader
模型:model
损失函数:loss_fn
优化器:optimizer

反向传播步骤：

清空参数梯度-计算参数的梯度-更新参数梯度


In [17]:
def train(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    X,y=X.to(device),y.to(device)

    pred=model(X)
    loss=loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch %100 ==0:
      loss,current=loss.item(),batch*len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## 5.定义测试函数

In [28]:
def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)

    model.eval()
    correct,loss=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()

    loss/=num_batches
    correct/=size
    print("loss:{},accuracy:{}".format(loss,correct))

## 6.训练


In [29]:
epochs=3
for epoch in range(epochs):
  print(f"Epoch {epoch+1}\n----------------------------")
  train(train_loader,model,loss_fn,optimizer)
  test(test_loader,model,loss_fn)
print("Done!")


Epoch 1
----------------------------
loss: 0.479604  [    0/  480]
loss: 0.270862  [  400/  480]
loss:0.488195055971543,accuracy:0.7583333333333333
Epoch 2
----------------------------
loss: 0.103656  [    0/  480]
loss: 0.275150  [  400/  480]
loss:0.4720036228497823,accuracy:0.75
Epoch 3
----------------------------
loss: 0.174424  [    0/  480]
loss: 0.097558  [  400/  480]
loss:0.4528446896622578,accuracy:0.7416666666666667
Done!
